In [ ]:

import os

import torch
import pickle
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import tensorflow as tf
from typing import Dict
from ast import literal_eval
from torch.optim import AdamW
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from torch.nn import BCEWithLogitsLoss, BCELoss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
from transformers import XLNetForSequenceClassification, XLNetTokenizer,BertForSequenceClassification,BertTokenizer, RobertaForSequenceClassification,RobertaTokenizer

In [ ]:
data = pd.read_csv("data train and test hashtag.csv")
data.head(2)

,Unnamed: 0,Title,Description,Topic,Descr
0,0,The Devil's Disciple,Set in Colonial America during the Revolutiona...,Fiction,set colonial america revolutionary era play te...
1,1,Taking Chances,Spice-o-meter Rating: This fun romance is a so...,Romance,spiceometer rating fun romance solid maybe sca...


In [ ]:
data = data.drop(["Unnamed: 0"], axis = 1)
data = data.dropna()
data.isna().sum()

In [ ]:
len(data)
X_test_pad =[]

18091

In [ ]:
from collections import Counter

topic_counts = Counter(data['Topic'])
sorted_topics = sorted(topic_counts, key=topic_counts.get, reverse=True)
label_mapping = {topic: i for i, topic in enumerate(sorted_topics)}
data['Label'] = data['Topic'].map(label_mapping)


In [ ]:
data

,Title,Description,Topic,Descr,Label
0,The Devil's Disciple,Set in Colonial America during the Revolutiona...,Fiction,set colonial america revolutionary era play te...,0
1,Taking Chances,Spice-o-meter Rating: This fun romance is a so...,Romance,spiceometer rating fun romance solid maybe sca...,10
2,Verdi : The Story of the Little Boy who Loved ...,This time Tapper moves his focus to Italy in h...,Biography & Autobiography,time tapper moves focus italy voyage among gre...,3
3,Success,The contents of this volume originally appeare...,Business,contents volume originally appeared weekly art...,20
4,Triplanetary,"ONE MAN DISCOVERED THE TRUTH—The Fall of Rome,...",Fiction,one man discovered truththe fall rome wars rac...,0
...,...,...,...,...,...
18095,Journey to the East,This book tells the tale of a man who goes on ...,other_1,book tells tale man goes wonderful amazing jou...,1
18096,The Monk Who Sold His Ferrari: A Fable About F...,"Wisdom to Create a Life of Passion, Purpose, a...",Biography & Autobiography,wisdom create life passion purpose peace inspi...,3
18097,I Am that,This collection of the timeless teachings of o...,Philosophy,collection timeless teachings one greatest sag...,9
18098,The Berlin Phenomenology,Since the three volume edition ofHegel's Philo...,History,since three volume edition ofhegels philosophy...,6


In [ ]:
import numpy as np

# Tính số lượng nhãn duy nhất trong cột label
num_labels = len(data['Label'].unique())

# Tạo một array numpy với kích thước (số dòng, số lượng nhãn duy nhất)
one_hot_labels = np.zeros((len(data), num_labels))

# Đánh dấu những dòng tương ứng với nhãn của nó
for i, label in enumerate(data['Label']):
    one_hot_labels[i][label] = 1

# Thêm array one-hot vào DataFrame
data['one_hot_label'] = list(one_hot_labels)

# In ra DataFrame sau khi thêm cột one-hot
data.head()


,Title,Description,Topic,Descr,Label,one_hot_label
0,The Devil's Disciple,Set in Colonial America during the Revolutiona...,Fiction,set colonial america revolutionary era play te...,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Taking Chances,Spice-o-meter Rating: This fun romance is a so...,Romance,spiceometer rating fun romance solid maybe sca...,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Verdi : The Story of the Little Boy who Loved ...,This time Tapper moves his focus to Italy in h...,Biography & Autobiography,time tapper moves focus italy voyage among gre...,3,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Success,The contents of this volume originally appeare...,Business,contents volume originally appeared weekly art...,20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Triplanetary,"ONE MAN DISCOVERED THE TRUTH—The Fall of Rome,...",Fiction,one man discovered truththe fall rome wars rac...,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Viết các thứ cần thiết cho bert model

In [ ]:
labels = list(data.one_hot_label.values)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Assuming you have 'data' dataframe containing your dataset

# Tokenizing the descriptions
max_length = 64
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

input_ids = []
attention_masks = []

for descr in data['Descr']:
    encoded_dict = tokenizer.encode_plus(
                        descr,
                        add_special_tokens=True,
                        max_length=64,
                        pad_to_max_length=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(np.stack(data['one_hot_label'].values))

# Splitting the data into train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.1)

# Creating DataLoader for train and validation sets
batch_size = 2

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

# Model

In [ ]:
from transformers import BertForSequenceClassification, AdamW

# Khởi tạo mô hình BERT cho phân loại chuỗi
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print('Model Pushed to Cuda for Training')

# Thiết lập tối ưu hóa AdamW với tốc độ học lr=6e-6
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=6e-6)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Model Pushed to Cuda for Training


In [ ]:
# train_loss_set = []
# val_f1_accuracy_list,val_flat_accuracy_list,training_loss_list,epochs_list=[],[],[],[]
y_test =[]
# # Number of training epochs (recommend between 5 and 10)
# epochs = 6

# # trange is a tqdm wrapper around the normal python range
# for _ in trange(epochs, desc="Epoch "):
#     # Training

#     # Set our model to training mode (as opposed to evaluation mode)
#     model.train()

#     # Tracking variables
#     tr_loss = 0 #running loss
#     nb_tr_examples, nb_tr_steps = 0, 0

#     # Train the data for one epoch
#     for step, batch in enumerate(train_dataloader):
#         # Add batch to GPU
#         batch = tuple(t.to(device) for t in batch)
#         # Unpack the inputs from our dataloader
#         b_input_ids, b_input_mask, b_labels= batch
#         # Clear out the gradients (by default they accumulate)
#         optimizer.zero_grad()

#         # Forward pass for multilabel classification
#         # https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html
#         # https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
#         # Creates a criterion that measures the Binary Cross Entropy between the target and the input probabilities
#         # Also This loss combines a Sigmoid layer and the BCELoss in one single class. This version is more numerically stable
#         # than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the
#         # log-sum-exp trick for numerical stability.
#         outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
#         logits = outputs[0]
#         loss_func = BCEWithLogitsLoss()
#         loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation

#         train_loss_set.append(loss.item())

#         # Backward pass
#         loss.backward()
#         # Update parameters and take a step using the computed gradient
#         optimizer.step()
#         # scheduler.step()
#         # Update tracking variables
#         tr_loss += loss.item()
#         nb_tr_examples += b_input_ids.size(0)
#         nb_tr_steps += 1

#     print("Train loss: {}".format(tr_loss/nb_tr_steps))
#     training_loss_list.append(tr_loss/nb_tr_steps)

#     ###############################################################################

#     # Validation

#     # Put model in evaluation mode to evaluate loss on the validation set
#     model.eval()

#     # Variables to gather full output
#     logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

#     # Predict
#     for i, batch in enumerate(validation_dataloader):
#         batch = tuple(t.to(device) for t in batch)
#         # Unpack the inputs from our dataloader
#         b_input_ids, b_input_mask, b_labels = batch
#         with torch.no_grad():
#             # Forward pass
#             outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
#             b_logit_pred = outs[0]
#             pred_label = torch.sigmoid(b_logit_pred)

#         b_logit_pred = b_logit_pred.detach().cpu().numpy()
#         pred_label = pred_label.to('cpu').numpy()
#         b_labels = b_labels.to('cpu').numpy()

#     tokenized_texts.append(b_input_ids)
#     logit_preds.append(b_logit_pred)
#     true_labels.append(b_labels)
#     pred_labels.append(pred_label)

#     # Flatten outputs
#     pred_labels = [item for sublist in pred_labels for item in sublist]
#     true_labels = [item for sublist in true_labels for item in sublist]

#     # Calculate Accuracy
#     threshold = 0.50
#     pred_bools = [pl>threshold for pl in pred_labels]
#     true_bools = [tl==1 for tl in true_labels]
#     val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
#     val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

#     print('F1 Validation Accuracy: ', val_f1_accuracy)
#     print('Flat Validation Accuracy: ', val_flat_accuracy)
#     print('\n')
#     val_f1_accuracy_list.append(val_f1_accuracy)
#     val_flat_accuracy_list.append(val_flat_accuracy)
#     epochs_list.append(epochs)

In [ ]:
from tqdm import tqdm

# Set gradient accumulation steps
gradient_accumulation_steps = 4
train_loss_values = []
val_accuracy_values = []
epochs = 3  # Số lượng epoch

for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss

        # Calculate total loss
        total_train_loss += loss.item() / gradient_accumulation_steps

        # Backpropagation
        loss.backward()

        # Gradient accumulation
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            model.zero_grad()

    avg_train_loss = total_train_loss / len(train_dataloader)
    train_loss_values.append(avg_train_loss)

    # Evaluation
    model.eval()
    total_eval_accuracy = 0

    with torch.no_grad():
        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs.logits
            probs = torch.sigmoid(logits)
            total_eval_accuracy += flat_accuracy(probs.cpu().numpy(), b_labels.cpu().numpy())

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    val_accuracy_values.append(avg_val_accuracy)

    print(f'Epoch {epoch + 1}')
    print(f'Average training loss: {avg_train_loss}')
    print(f'Validation Accuracy: {avg_val_accuracy}')

print("Training complete.")


In [ ]:
def predict(X_test, model, device, batch_size=32):
    """
    Predict labels for test data using the trained model.

    Args:
    X_test (torch.Tensor): Test input data.
    model (torch.nn.Module): Trained model.
    device (torch.device): Device to perform predictions on (e.g., 'cuda' or 'cpu').
    batch_size (int, optional): Batch size for prediction. Defaults to 32.

    Returns:
    np.ndarray: Predicted labels for the test data.
    """

    model.eval()
    predictions = []

    test_dataloader = DataLoader(X_test, batch_size=batch_size)

    with torch.no_grad():
        for batch in test_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask = batch
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs.logits
            probs = torch.sigmoid(logits)
            batch_predictions = probs.cpu().numpy()
            predictions.extend(batch_predictions)

    return np.array(predictions)
predictions = predict(X_test_pad, model, device)

def calculate_accuracy(predictions, true_labels, threshold=0.5):
    """
    Calculate accuracy based on predictions and true labels.

    Args:
    predictions (np.ndarray): Predicted probabilities or scores.
    true_labels (np.ndarray): True labels.
    threshold (float, optional): Threshold for binary classification. Defaults to 0.5.

    Returns:
    float: Accuracy score.
    """

    # Làm tròn dự đoán về 0 hoặc 1 dựa trên ngưỡng
    predicted_labels = (predictions >= threshold).astype(int)

    # Tính độ chính xác
    accuracy = np.mean(predicted_labels == true_labels)

    return accuracy
acc=calculate_accuracy(predictions, y_test, threshold=0.5)
print(acc)

0.6045877


In [ ]:
train_generator = iter(train_dataloader)
batch = next(train_generator)
print(len(batch))


In [ ]:
import gc
gc.collect()